## 准备数据

In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, optimizers, datasets

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # or any {'0', '1', '2'}

def mnist_dataset():
    (x, y), (x_test, y_test) = datasets.mnist.load_data()
    #normalize
    x = x/255.0
    x_test = x_test/255.0
    
    return (x, y), (x_test, y_test)

In [2]:
print(list(zip([1, 2, 3, 4], ['a', 'b', 'c', 'd'])))

[(1, 'a'), (2, 'b'), (3, 'c'), (4, 'd')]


## 建立模型

In [6]:
class myModel:
    def __init__(self):
        ####################
        '''声明模型对应的参数'''
        ####################
        self.W1 = tf.Variable(tf.random.normal([784, 256]), name='W1')
        self.b1 = tf.Variable(tf.zeros([256]), name='b1')
        self.W2 = tf.Variable(tf.random.normal([256, 10]), name='W2')
        self.b2 = tf.Variable(tf.zeros([10]), name='b2')
    def __call__(self, x):
        ####################
        '''实现模型函数体，返回未归一化的logits'''
        ####################
        x = tf.reshape(x, [-1, 784])
        h = tf.nn.relu(tf.matmul(x, self.W1) + self.b1)
        logits = tf.matmul(h, self.W2) + self.b2
        return logits
        
model = myModel()

optimizer = optimizers.Adam()

## 计算 loss

In [15]:
@tf.function
def compute_loss(logits, labels):
    return tf.reduce_mean(
        tf.nn.sparse_softmax_cross_entropy_with_logits(
            logits=logits, labels=labels))

@tf.function
def compute_accuracy(logits, labels):
    predictions = tf.argmax(logits, axis=1)
    return tf.reduce_mean(tf.cast(tf.equal(predictions, labels), tf.float32))

@tf.function
def train_one_step(model, optimizer, x, y):
    with tf.GradientTape() as tape:
        logits = model(x)
        loss = compute_loss(logits, y)

    # compute gradient
    trainable_vars = [model.W1, model.W2, model.b1, model.b2]
    grads = tape.gradient(loss, trainable_vars)
    for g, v in zip(grads, trainable_vars):
        v.assign_sub(0.01*g)

    accuracy = compute_accuracy(logits, y)

    # loss and accuracy is scalar tensor
    return loss, accuracy

@tf.function
def test(model, x, y):
    logits = model(x)
    loss = compute_loss(logits, y)
    accuracy = compute_accuracy(logits, y)
    return loss, accuracy

## 实际训练

In [16]:
train_data, test_data = mnist_dataset()
for epoch in range(50):
    loss, accuracy = train_one_step(model, optimizer, 
                                    tf.constant(train_data[0], dtype=tf.float32), 
                                    tf.constant(train_data[1], dtype=tf.int64))
    print('epoch', epoch, ': loss', loss.numpy(), '; accuracy', accuracy.numpy())
loss, accuracy = test(model, 
                      tf.constant(test_data[0], dtype=tf.float32), 
                      tf.constant(test_data[1], dtype=tf.int64))

print('test loss', loss.numpy(), '; accuracy', accuracy.numpy())

epoch 0 : loss 7.4839125 ; accuracy 0.81145
epoch 1 : loss 7.478818 ; accuracy 0.81151664
epoch 2 : loss 7.473734 ; accuracy 0.81155
epoch 3 : loss 7.4686604 ; accuracy 0.81156665
epoch 4 : loss 7.463597 ; accuracy 0.81163335
epoch 5 : loss 7.458544 ; accuracy 0.8117333
epoch 6 : loss 7.4535007 ; accuracy 0.8118167
epoch 7 : loss 7.4484677 ; accuracy 0.8119
epoch 8 : loss 7.4434443 ; accuracy 0.812
epoch 9 : loss 7.4384303 ; accuracy 0.8121
epoch 10 : loss 7.4334254 ; accuracy 0.8121833
epoch 11 : loss 7.4284296 ; accuracy 0.81233335
epoch 12 : loss 7.4234443 ; accuracy 0.8124
epoch 13 : loss 7.418469 ; accuracy 0.8124667
epoch 14 : loss 7.413503 ; accuracy 0.8125167
epoch 15 : loss 7.408546 ; accuracy 0.81256664
epoch 16 : loss 7.4035993 ; accuracy 0.81268334
epoch 17 : loss 7.398663 ; accuracy 0.8127667
epoch 18 : loss 7.393736 ; accuracy 0.8129333
epoch 19 : loss 7.3888187 ; accuracy 0.81303334
epoch 20 : loss 7.3839116 ; accuracy 0.8131
epoch 21 : loss 7.379013 ; accuracy 0.8131667